In [1]:
import requests
import json
import httpx
from typing import Optional, Dict, Any
import asyncio
from src.utils.api_reader import OASParser
from pprint import pprint
from src.dynamic_requests import URESTAdapter
from kafka import KafkaProducer


Инициализация Продюссера

In [3]:
BOOTSTRAP_SERVERS = ['localhost:9092']  # или ваш сервер Kafka
producer = KafkaProducer(
    bootstrap_servers=BOOTSTRAP_SERVERS,
    value_serializer=lambda v: json.dumps(v).encode('utf-8'),
    # Дополнительные параметры:
    acks='all',  # подтверждение от всех реплик
    retries=3,    # количество попыток
    max_in_flight_requests_per_connection=1,
    compression_type='gzip'  # сжатие
)


NoBrokersAvailable: NoBrokersAvailable

In [ ]:
Определяем сущность (пока временно это operationID),он же будет топиком в кафку

In [5]:
TOPIC_NAME = 'getEverything'
#  forecast - Яндекс Погода
#  getTopHeadlines - API новостной с пагинацей
#  getEverything - API новостной с пагинацей
#  suggestBank - Dadata - сервис API

src_dict = {'getEverything':'C:/Users/kdenis/Documents/Work/OpenApiSpecParser/examples/NewsAPI.yaml'
            ,'suggestBank':'C:/Users/kdenis/Documents/Work/OpenApiSpecParser/examples/suggestions.yml'
            ,'getTopHeadlines':'C:/Users/kdenis/Documents/Work/OpenApiSpecParser/examples/NewsAPI.yaml'
            ,'getCurrentConditions':'C:/Users/kdenis/Documents/Work/OpenApiSpecParser/examples/accuweather.yaml'}

Новостная:

Swift-банков

In [6]:
input_data = {
    'getEverything':{'q':'RUSSIA'},
 'suggestBank':[
    'SABRRUMM', 'VTBRRUMM', 'GAZPRUMM', 'ALFARUMM', 'MOSCRUMM',
    'RSBNRUMM', 'RUWCRUMM', 'ICBKRUMM', 'KOSKRUMM', 'PARNRUMM',
    'ABNYRUMM', 'CRYPRUMM', 'TICSRUMM', 'PSBZRUMM', 'TKRBRUMM',
    'JSNMRUMM', 'MIRBRUMM', 'ELSRUMMXXX', 'RNGBRUMM', 'IRONRUMM',
    'AVJSRUMM', 'ARESRUMM', 'ALILRUMM', 'ITRORU8Y', 'BOCSRUMM',
    'DOMRRUMM', 'FORTRUMM', 'GLBKRUMM', 'HDCBRUMM', 'KBKTRUMM',
    'KREMRUMM', 'LBRURUMM', 'MDMBRUMM', 'MEZHRUMM', 'MOPARUMM',
    'OLMDRUMM', 'ROYCRUMM', 'RZCBRUMM', 'SBERRUMM', 'SGBZRUMM',
    'SLAVRUMM', 'SOGZRUMM', 'TATKRUMM', 'TKBKRUMM', 'TKZLRUMM',
    'TKZVRUMM', 'TRNVRUMM', 'VEFKRUMM', 'VTBKRUMM', 'ZENIRUMM'
],
'getTopHeadlines':{'q':'RUSSIA'},
'getCurrentConditions':[1,2,3,45,66666,77777,9999,88]
}

Инициализция Парсера + Адаптера - различные варианты:

In [7]:

parser = OASParser(src_dict[TOPIC_NAME])
entity  = parser.request.get(TOPIC_NAME)
pprint(entity)




{'auth_types': [{'apiKey': []}],
 'headers': {'Accept': 'application/json', 'Content-Type': 'application/json'},
 'method': 'GET',
 'operationalId': 'getEverything',
 'required': [],
 'url': 'https://newsapi.org/v2/everything',
 'variables': ['domains',
               'excludeDomains',
               'from',
               'language',
               'page',
               'pageSize',
               'q',
               'searchIn',
               'sortBy',
               'sources',
               'to']}


In [8]:
test = URESTAdapter(entity,'C:/Users/kdenis/mu_code/keys.json')
pprint(entity)


{'auth_types': [{'apiKey': []}],
 'headers': {'Accept': 'application/json', 'Content-Type': 'application/json'},
 'method': 'GET',
 'operationalId': 'getEverything',
 'required': [],
 'url': 'https://newsapi.org/v2/everything',
 'variables': ['domains',
               'excludeDomains',
               'from',
               'language',
               'page',
               'pageSize',
               'q',
               'searchIn',
               'sortBy',
               'sources',
               'to']}


In [10]:
result = test.execute(input_data.get(TOPIC_NAME))
pprint(result)


переменная(ые) есть в списке
[{'articles': [{'author': 'Matthew Gault',
                'content': 'For half a century, the worlds nuclear powers '
                           'relied on an intricate and complex series of '
                           'treaties that slowly and steadily reduced the '
                           'number of nuclear weapons on the planet. Those '
                           'treaties are go… [+3542 chars]',
                'description': 'The last major nuclear arms treaty between the '
                               'US and Russia just expired. Some experts '
                               'believe a combination of satellite '
                               'surveillance, AI, and human reviewers can take '
                               'its place. Others, not so much.',
                'publishedAt': '2026-02-09T11:30:00Z',
                'source': {'id': 'wired', 'name': 'Wired'},
                'title': 'AI Is Here to Replace Nuclear Treaties. Scared Yet

In [9]:
pprint(entity)
print('*'*40)


{'auth_types': None,
 'headers': {'Accept': 'application/json', 'Content-Type': 'application/json'},
 'method': 'POST',
 'operationalId': 'suggestBank',
 'required': ['query'],
 'url': 'https://suggestions.dadata.ru/suggestions/api/4_1/rs/suggest/bank',
 'variables': ['count',
               'locations',
               'locations[].kladr_id',
               'locations_boost',
               'locations_boost[].kladr_id',
               'query',
               'status',
               'type']}
****************************************


Обработка спецификации и создание файла конфигурации